In [1]:
import pandas as pd
df = pd.read_csv("data/store_code.csv", encoding='cp949')

In [2]:
# 구로구랑 은평구만 담음
df = df[(df['자치구_코드_명'] == '구로구') | (df['자치구_코드_명'] == '은평구')]

In [3]:
# 컬럼 정리
df = df[['상권배후지_구분_코드', '상권배후지_구분_코드_명', '자치구_코드', '자치구_코드_명']]

In [ ]:
# 컬럼 영문명으로 변환
df = df.rename(columns={
    '상권배후지_구분_코드': 'region_code',
    '상권배후지_구분_코드_명': 'region_name',
    '자치구_코드': 'district_code',
    '자치구_코드_명': 'district',
    '위도': 'latitude',
    '경도': 'longitude'
})

In [5]:
# Kakao REST API 키
api_key = "5a50fb64d03a043ff6d223cb53646925"

In [6]:
# latitude, longitude 컬럼 초기화
df['latitude'] = None
df['longitude'] = None

In [ ]:
import requests
import time
import re

rows_to_keep = []  # 검색 성공한 인덱스 저장

# 카카오맵 API
for idx, row in df.iterrows():
    place = row['region_name']

    # 괄호 안 내용 제거
    place = re.sub(r'\(.*?\)', '', place).strip()

    url = f"https://dapi.kakao.com/v2/local/search/keyword.json?query={place}"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    
    try:
        response = requests.get(url, headers=headers).json()
        if response['documents']:
            doc = response['documents'][0]  # 가장 첫번째 결과 사용
            df.at[idx, 'latitude'] = doc.get('y')
            df.at[idx, 'longitude'] = doc.get('x')
            rows_to_keep.append(idx)  # 성공한 인덱스 저장
        else:
            print(f"[Warning] '{place}' 검색 결과 없음")
    except Exception as e:
        print(f"[Error] '{place}' 처리 중 오류 발생: {e}")
    
    time.sleep(0.3)  # API 호출 제한 방지 (0.3~0.5초 권장)

[Warning] '큰굴다리' 검색 결과 없음
[Warning] '구로나눔길' 검색 결과 없음


In [8]:
# 검색 성공한 행만 남기기
df = df.loc[rows_to_keep].reset_index(drop=True)

In [9]:
# 5. 결과 엑셀로 저장
output_path = r'data/01_store_code.csv'
df.to_csv(output_path, index=False)
print("CSV 저장 경로:", output_path)

CSV 저장 경로: data/01_store_code.csv
